In [1]:
import numpy as np
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important }</style>"))

In [4]:
pd.options.display.max_columns = 1000

In [5]:
cust = pd.read_csv("CUST_DATA.csv", encoding="utf-16")
claim = pd.read_csv("CLAIM_DATA.csv", encoding="utf-16")

# 1. 여기부터 전처리 코드를 작성하세요.

# 여기부터는 수정하면 안됩니다.

# 전처리 완료된 파일을 저장하세요. 

In [133]:
cust.to_csv("CUST_DATA_전처리완료.csv", index=False, encoding="utf-8-sig")

In [134]:
cust.shape

(22400, 92)

### 전처리 완료된 데이터를 불러와서 훈련 데이터와 검증데이터로 나눔

In [153]:
cust = pd.read_csv("CUST_DATA_전처리완료.csv", encoding="utf-8-sig")

In [154]:
train = cust[cust['DIVIDED_SET']==1].reset_index(drop=True)
test = cust[cust['DIVIDED_SET']==2].reset_index(drop=True)

In [155]:
train.drop('DIVIDED_SET', axis=1, inplace=True)
test.drop('DIVIDED_SET', axis=1, inplace=True)

In [156]:
train_X = train.drop(['CUST_ID', 'SIU_CUST_YN'], axis=1)
train_y = train['SIU_CUST_YN']

In [157]:
test_X = test.drop(['CUST_ID', 'SIU_CUST_YN'], axis=1)

In [158]:
train['SIU_CUST_YN'].value_counts()

0.0    18801
1.0     1806
Name: SIU_CUST_YN, dtype: int64

In [159]:
train_X.shape, train_y.shape

((20607, 89), (20607,))

### Random Forest를 이용한 모델링

In [160]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=100, max_features=16, random_state=42)
rf_model.fit(train_X, train_y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=16,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [161]:
y_pred = rf_model.predict(test_X)

In [162]:
y_pred

array([0., 0., 0., ..., 0., 0., 0.])

In [169]:
val_df = pd.read_csv("answer.csv") # 모평을 평가하기 위한 정답을 불러옴

In [170]:
val_df.head()

,CUST_ID,YN
0,37,0
1,51,0
2,60,0
3,65,0
4,73,0


In [171]:
pd.crosstab(val_df.YN, y_pred) # 교차분류표

col_0,0.0,1.0
YN,,
0,1571,64
1,152,6


In [172]:
rf_model.score(test_X, val_df.YN)

0.879531511433352

### 10회 교차검증 후 f1 스코어의 평균을 출력합니다. 
### 출력한 값이 가장 큰 팀이 우승

In [173]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(rf_model, test_X, val_df.YN , cv=10, scoring='f1_macro')
scores.mean()

0.47680068420348815